# Data Cleaning and Preparation 
### Handling Missing Data
all the descriptive statistics on pandas objects exclude missing data by default. Numeric missing data is represented by NaN. This is called the sentinal value

In [ ]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [ ]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

In [ ]:
string_data.isnull() # returns a boolean for data that are missing

In [ ]:
# assign first index
string_data[0] = None # built in None value is treated as a NA value
string_data.isnull()

In [ ]:
string_data.dropna() # returns the values that are not missing

In [ ]:
string_data.fillna('blank') # fills missing value with a string

### Filtering Out Missing Data

In [ ]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data[data.notnull()] # filter out the missing vaues

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna() # drops al rows with any missing values
data
cleaned

In [ ]:
data.dropna(how='all') # drops rows that are all NA

In [ ]:
data[4] = NA # add a column wth NAs
data.dropna(axis=1, how='all') # drops columns that are all NA, when we specify axix=1

In [ ]:
# we want ot keep only rows containing a certain number of observations
df = pd.DataFrame(np.random.randn(7,3))
df.iloc[:4,1] = NA # indexes 0 through 4, excluding 4, and column index 1, second column
df.iloc[:2,2] = NA # indexes 0 through 2 excluding2, ad column index 2, third column
df



In [ ]:
df.dropna(thresh=2) # the minimum number of non-NAN values needed in a row not to drop. Threshold = 2 means that a column as at least 2 non-NaNs to survive.

### Filling in Missing Data
fillna('value') fills in every missing value with the string. fillna returns a new object

In [ ]:
# calloing fillna with a dict allows us ot use different fill values for each column
df.fillna({1: 0.5, 2: 0}) # fills the 1 index, second column with 0.5 and fills third column with 0

In [ ]:
# fillna returns a new object but we can modify the exsiting object in-place
df.fillna(0, inplace=True)

In [ ]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df.fillna(method='ffill') # foward fills the NA with the first vlaue from the top
df.fillna(method='ffill', limit=2) # limits the foward fill to only two entries

In [ ]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean()) # fills missing vlaues with the mean

### Data Transformation

In [ ]:
# removing duplicates
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

In [ ]:
data.duplicated() # returns true for all data points in the row that are duplicates

In [ ]:
data.drop_duplicates() # drops al duplicates

In [ ]:
data['v1']= range(7) # add a column
data.drop_duplicates('k1') # drops duplicates based on only the k1 column

In [ ]:
data.drop_duplicates(['k1', 'k2'], keep='last')

### Transforming data using a function or mapping

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [ ]:
# to add a column indicating the type of animal that each food came from, we first need a mapping of each distinct meat type to the dind of animal
# this serves as a look up table
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [ ]:
# to map , we first need to convert everything to lower case
lowercased = data['food'].str.lower()
lowercased
data['animal'] = lowercased.map(meat_to_animal) # adds a column by mapping the dict. The keys are in lowercase, the values are what is assigned to the column
data

In [ ]:
data['food'].map(lambda x: meat_to_animal[x.lower()]) # also can use a lambda function. This returms on value

### Replacing values   

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

In [ ]:
# the -999 may be considered sential values
data.replace(-999, np.nan)# replaces -999 values witj NA values as a new series. Pass inplace=True to not make a new series with the environment

In [ ]:
# to replace multiple vlaues at once
data.replace([-999, -1000], np.nan) # pass a list

In [ ]:
# to use a different replacement values pass a list of substitutes
data.replace([-999, -1000], [np.nan, 0])

In [ ]:
data.replace({-999: np.nan, -1000: 0}) # can also pass a dict for different replacement values

### Renaming Axis Indexes  
can be done inpace or not

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

In [ ]:
transform = lambda x: x[:4].upper() # create a function to grab the first 4 indexes (0 to 3) and convert to upper
# like a Series, axis indexes have a map method to rename axis indexes
data.index.map(transform)

In [ ]:
# to modify the DataFrame in place assign to index
data.index = data.index.map(transform)
data

In [ ]:
# to create a transformed version of a dataset without modifying the original, use rename
data.rename(index=str.title, columns=str.upper) # converts indexes to title case and colum ns to upper case


In [ ]:
# use a dict to provide a subset of axis labels (use inplace = True if needed)
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})


### Discretization and Binning
convert continuous data to discrete values or binning continuous data

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100] # createe bins 18 to 25, 26 to 35, 36 to 60
cats = pd.cut(ages, bins) # this returns a Category object. Treat as an array of strings indicating the bin name
cats

In [ ]:
cats.codes # returns index codes of each category from the discrete bins of ages

In [ ]:
cats.categories # returns defines catagories of the Category object

In [ ]:
pd.value_counts(cats) # count the values that are in each of the categories. They are the bin counts from the result of pandas.cut


In [ ]:
# a ( means that the side is open. a } means it is closed inclusive. WE can change which side is closed by passing right=False)
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior'] # define a group of bin names
pd.cut(ages, bins, labels=group_names) # assign lables to the bins

In [ ]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2) # passes an integer number of bins instead of an explicit bin list. computes equal legth bins based on the min and max values in the data
# the precision = 2 optines limits the decimal precision to two digits

In [ ]:
# depending on the data distribution, pd.cut may not result in each bin having the same number of data points
data= np.random.randn(1000) # normally distributed data
cats = pd.qcut(data, 4)
cats

In [ ]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]) # this allows use to specify our own quantiles between 0 and 1

### Detecting and Filtering Outliers

In [ ]:
data = pd.DataFrame(np.random.randn(1000, 4)) # creates a matrix of 1000 rows and 4 columns
data.describe()


In [ ]:
col = data[2] # grabs the third column
col[np.abs(col) > 3] # retunrs the abslute values > 3

In [ ]:
data[(np.abs(data) > 3).any(1)] # selects all rows having a vlaue exceding 3 or -3

In [ ]:
np.sign(data).head() # sign produces 1 and -1 values based on whether the values are positive or negative

### Permutation and Random Sampling
permuting is randomly reordering

In [101]:
df = pd.DataFrame(np.arange(5*4).reshape((5, 4))) # 5*4 instead of 20 just allows us to not figure our how many values we need
sampler = np.random.permutation(5) # creates an array of randomly reordered (permutations) of integeres between 0 and 4
sampler  
df.take(sampler) # selects rows based on the sampler array returning a reordered dataframe

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
3,12,13,14,15
4,16,17,18,19
2,8,9,10,11


In [102]:
df.sample(n=3) # selects a random subet without replacement of the specified rows (n)

,0,1,2,3
2,8,9,10,11
1,4,5,6,7
4,16,17,18,19


In [104]:
choices = pd.Series([5,7,-1,6,4])
draws = choices.sample(n=10, replace=True) # generates a sample with replacement (to allow repeat choices)
draws

3    6
4    4
2   -1
3    6
1    7
4    4
0    5
2   -1
2   -1
4    4
dtype: int64

### Computing Indicator/Dummy Variables
converting categorial variables into dummy variables

In [106]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [107]:
pd.get_dummies(df['key']) # create the dummy variables for the 'key' column

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [109]:
dummies = pd.get_dummies(df['key'], prefix='key') # inserts a prefix in front of the dummy variable to help distinguish it as a dummy variable
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [111]:
df_with_dummy = df[['data1']].join(dummies) # grabs the 'data1' column from data datafraem ad joins the dummy variables to it
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [118]:
# is a row in a DataFrame belongs to multiple categories, things get more compicated
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10] # shows rows 0 through 9 indexes

C:\Users\alexm\AppData\Local\Temp\ipykernel_51296\2919338712.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/movielens/movies.dat', sep='::',


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [120]:
# our goal is to create dummy variables for each genre. Given that movies can have multiple genres, we have to do some wrangling
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|')) # extend adds a list of things to extend to the list. The slit function creates a list of items delimited by the '|'
all_genres # this has duplicates, we need a unique list

['Animation',
 "Children's",
 'Comedy',
 'Adventure',
 "Children's",
 'Fantasy',
 'Comedy',
 'Romance',
 'Comedy',
 'Drama',
 'Comedy',
 'Action',
 'Crime',
 'Thriller',
 'Comedy',
 'Romance',
 'Adventure',
 "Children's",
 'Action',
 'Action',
 'Adventure',
 'Thriller',
 'Comedy',
 'Drama',
 'Romance',
 'Comedy',
 'Horror',
 'Animation',
 "Children's",
 'Drama',
 'Action',
 'Adventure',
 'Romance',
 'Drama',
 'Thriller',
 'Drama',
 'Romance',
 'Thriller',
 'Comedy',
 'Action',
 'Action',
 'Comedy',
 'Drama',
 'Crime',
 'Drama',
 'Thriller',
 'Thriller',
 'Drama',
 'Sci-Fi',
 'Drama',
 'Romance',
 'Drama',
 'Drama',
 'Romance',
 'Adventure',
 'Sci-Fi',
 'Drama',
 'Drama',
 'Drama',
 'Sci-Fi',
 'Adventure',
 'Romance',
 "Children's",
 'Comedy',
 'Drama',
 'Drama',
 'Romance',
 'Drama',
 'Documentary',
 'Comedy',
 'Comedy',
 'Romance',
 'Drama',
 'Drama',
 'War',
 'Action',
 'Crime',
 'Drama',
 'Drama',
 'Action',
 'Adventure',
 'Comedy',
 'Drama',
 'Drama',
 'Romance',
 'Crime',
 'Thrill

In [123]:
genres = pd.unique(all_genres)
genres

18

In [126]:
# to create the dummy DataFrame, start with a DataFrae of all zeros
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres) # assign the genres names to the columns of the zero matirx
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
# now iterate throuhg each movie and set entries in each row of dummies to 1
gen = movies.genres[0] # the first entrie or first row in the movies dataframe
gen.split('|')

['Animation', "Children's", 'Comedy']

In [139]:
dummies.columns.get_indexer(gen.split('|')) # the get_indexer finds the set of indexes for the list in the parameter. 
#the split return a list of genres. The get_indexer finds the column index for each genre in this list.
movies.genres

0        Animation|Children's|Comedy
1       Adventure|Children's|Fantasy
2                     Comedy|Romance
3                       Comedy|Drama
4                             Comedy
                    ...             
3878                          Comedy
3879                           Drama
3880                           Drama
3881                           Drama
3882                  Drama|Thriller
Name: genres, Length: 3883, dtype: object

In [134]:
# use .iloc to set values based on these indicies
# iterate through each row i, while getting each set of genres from the get_indexer and assign value of 1 to each row, i and each column index from the get_indexer list
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1 # set entries in row to 1
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
# combine the dummies with the movies
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.head()

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [145]:
# we can combine get_dummies with a dicretization function like cut
np.random.seed(12345)
values = np.random.rand(10)
bins = [0, 0.2, 0.4, 0.6, 0.8, 1] # create bins
pd.get_dummies(pd.cut(values, bins)) # creates dummy variables based on the values assignment in each bin

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## String Manipulation
### String Object Methods

In [146]:
val = 'a,b,  guido'
val.split(',') # creaes a list of items split by the comma

['a', 'b', '  guido']

In [148]:
# strip trims the white space
pieces = [x.strip() for x in val.split(',')] # list comprehension to apply the strip function to the val list
pieces

['a', 'b', 'guido']

In [149]:
# substrings can be concatonated together
first, second, third = pieces
first + '::' + second + "::" + third

'a::b::guido'

In [150]:
'::'.join(pieces) # a better way is to pass a list or tuple to the join method on the string '::""

'a::b::guido'

In [151]:
"guido" in val # checks if string is in list

True

In [152]:
val.index(',') # returns the index of the substring

1

In [156]:
val.find(':') # returns the index of the substring. A -1 means is did not find it

-1

In [159]:
val.count(',') # returns the count of the number of occurences of the substring

2

In [160]:
val.replace(',', '::') # replaces ',' with the '::'

'a::b::  guido'

In [161]:
val.replace(',', '') # used to delete patterns. deletes the commas

'ab  guido'

### Regular Expression
regex . Python uses the re module. fall in three categories: pattern matching, substitution, and splitting

In [162]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text) # splits on one or more whitespaces

['foo', 'bar', 'baz', 'qux']

In [163]:
regex = re.compile('\s+') # forms a reusable regex object
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [164]:
regex.findall(text)  # finds all patterns with one or more whitespaces '\s+'

['    ', '\t ', '  \t']

In [165]:
# findall returns all matches. Search returns only the first match

text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}' # a regex that can find nearly all emails

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE) # forms a reusable regex object with the given case-insensitive pattern

In [166]:
regex.findall(text) # finds all emails

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [168]:
m = regex.search(text) # finds the first match. Rerturns a special match bject
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [169]:
text[m.start():m.end()] # selects substring based on start and end values of m

'dave@google.com'

In [170]:
print(regex.match(text)) # only will match if the pattern occurs at the start of the string

None


In [172]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})' # to segment searches, put a () around the parts of the pattern to segment to form groups
regex = re.compile(pattern, flags=re.IGNORECASE)

In [173]:
regex.findall(text) # returns a list of tuples when the patterns has groups designated by the () in the regex

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [174]:
# \1 is grup 1, \2 is group 2 and so on
print(regex.sub(r'Username: \1. Domain: \2, Suffix: \3', text))

Dave Username: dave. Domain: google, Suffix: com
Steve Username: steve. Domain: gmail, Suffix: com
Rob Username: rob. Domain: gmail, Suffix: com
Ryan Username: ryan. Domain: yahoo, Suffix: com



### Vectorized String Functions in pandas

In [178]:
# a column with missing data
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [179]:
# we can use lambda functions an map but will fail on NA(null) values. Series has array-oriencted methods to skip NA values
data.str.contains('gmail') # returns a booleand indicating if there is a 'gmail' or not


Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [180]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
data.str.findall(pattern, flags=re.IGNORECASE) 

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [182]:
# element retrieval
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [184]:
matches.str.get(1) # gets the second index 

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

In [185]:
data.str[:5] # slices the 0 through 4 indexes

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [186]:
data.str.extract(pattern, flags=re.IGNORECASE) # returns the captured groups of a regex as a DataFrame

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN
